In [1]:
!pip install xlsxwriter pydrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 9.3 MB/s eta 0:00:00
  Created wheel for pydrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27433 sha256=9331887b11b38668cce0850c9024f0a07c9ffd636f7ee1be4dc67f9df36166bd
  Stored in directory: /root/.cache/pip/wheels/6c/10/da/a5b513f5b3916fc391c20ee7b4633e5cf3396d570cdd74970f
Successfully built pydrive


In [ ]:
#%% import
from __future__ import unicode_literals
import re
import time
from datetime import datetime
from time import mktime
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import json
import xlsxwriter


#%% functions - data conversion

def date_change_format(date_string):
    """
    Konwertuje datę z różnych formatów na "YYYY-MM-DD"
    """
    try:
        date_string = ' '.join(date_string.strip().split())

        if re.match(r'\d{4}-\d{2}-\d{2}', date_string):
            return date_string[:10]

        if 'T' in date_string:
            return date_string.split('T')[0]

        if re.match(r'\d{2}\.\d{2}\.\d{4}', date_string):
            result = time.strptime(date_string, "%d.%m.%Y")
            changed_date = datetime.fromtimestamp(mktime(result))
            return format(changed_date.date())

        lookup_table = {
            "stycznia": "01", "lutego": "02", "marca": "03", "kwietnia": "04",
            "maja": "05", "czerwca": "06", "lipca": "07", "sierpnia": "08",
            "września": "09", "października": "10", "listopada": "11", "grudnia": "12",
            "styczeń": "01", "luty": "02", "marzec": "03", "kwiecień": "04",
            "maj": "05", "czerwiec": "06", "lipiec": "07", "sierpień": "08",
            "wrzesień": "09", "październik": "10", "listopad": "11", "grudzień": "12"
        }

        for k, v in lookup_table.items():
            date_string = date_string.replace(k, v)

        if re.match(r'\d{1,2}\.\d{1,2}\.\d{4}', date_string):
            result = time.strptime(date_string, "%d.%m.%Y")
        else:
            result = time.strptime(date_string, "%d %m %Y")

        changed_date = datetime.fromtimestamp(mktime(result))
        return format(changed_date.date())
    except Exception as e:
        return "no date"


#%% functions - link extraction

def get_all_article_links():
    """
    Pobiera wszystkie linki do artykułów z portalkryminalny.pl
    Struktura: <div class="news" onclick="location.href='aktualnosci/kategoria/tytul'">
    """
    base_url = "https://portalkryminalny.pl"

    print("="*60)
    print("KROK 1: Pobieranie linków do artykułów")
    print("="*60 + "\n")

    all_links = set()

    # Kategorie do scrapowania
    categories = {
        'wszystkie': '/aktualnosci',
        'nowości': '/aktualnosci/nowosci',
        'zapowiedzi': '/aktualnosci/zapowiedzi',
        'wydarzenia': '/aktualnosci/wydarzenia',
        'wywiady': '/aktualnosci/wywiady',
        'recenzje': '/aktualnosci/recenzje',
    }

    for cat_name, cat_url in categories.items():
        print(f"Pobieranie linków z kategorii: {cat_name}...")

        # portalkryminalny.pl ma paginację - próbujemy pobrać kilka pierwszych stron
        for page in range(1, 6):  # pierwsze 5 stron każdej kategorii
            try:
                if page == 1:
                    url = base_url + cat_url
                else:
                    # Paginacja to /strona/2, /strona/3 itd.
                    url = f"{base_url}{cat_url}/strona/{page}"

                r = requests.get(url, timeout=10)
                r.encoding = 'utf-8'

                if r.status_code == 200:
                    soup = BeautifulSoup(r.text, 'lxml')

                    # Szukamy divów z klasą "news" które mają onclick z linkiem
                    # <div class="news" onclick="location.href='aktualnosci/nowosci/lowca-aurelia-es'">
                    news_divs = soup.find_all('div', class_='news')

                    page_links = 0
                    for div in news_divs:
                        onclick = div.get('onclick', '')
                        # Wyciągamy link z onclick="location.href='...'"
                        match = re.search(r"location\.href='([^']+)'", onclick)
                        if match:
                            href = match.group(1)

                            # Upewniamy się, że to pełny URL
                            if href.startswith('http'):
                                full_url = href
                            elif href.startswith('/'):
                                full_url = base_url + href
                            else:
                                full_url = base_url + '/' + href

                            # Dodajemy tylko linki do artykułów (nie do stron kategorii)
                            if '/aktualnosci/' in full_url and '/strona/' not in full_url:
                                all_links.add(full_url)
                                page_links += 1

                    print(f"  Strona {page}: {page_links} linków")

                    if page_links == 0:
                        break  # Jeśli nie ma więcej artykułów, przerywamy paginację

                    time.sleep(0.5)  # Aby nie przeciążać serwera

                else:
                    print(f"  ✗ Status {r.status_code} dla strony {page}")
                    break

            except Exception as e:
                print(f"  ✗ Błąd na stronie {page}: {e}")
                break

        print(f"  ✓ Zebrano linki z kategorii {cat_name}\n")

    # Konwertujemy set na listę
    all_links = list(all_links)

    print(f"{'='*60}")
    print(f"Łącznie znaleziono: {len(all_links)} unikalnych artykułów")
    print(f"{'='*60}\n")

    return all_links


#%% functions - scraping

def dictionary_of_article(article_link):
    """
    Pobiera szczegóły artykułu z portalkryminalny.pl
    """
    try:
        r = requests.get(article_link, timeout=15)
        r.encoding = 'utf-8'

        if r.status_code != 200:
            errors.append(article_link)
            return

        html_text = r.text

        if 'error 404' in html_text.lower() or 'page not found' in html_text.lower():
            errors.append(article_link)
            return

        soup = BeautifulSoup(html_text, 'lxml')

        # Data publikacji
        try:
            # W HTML: <p class="author">Autor: <span>materiały wydawcy</span> <br class="visible-xs">Data publikacji: <span>13 stycznia 2026</span></p>
            author_p = soup.find('p', class_='author')
            if author_p:
                date_text = ''
                # Szukamy tekstu po "Data publikacji:"
                text = author_p.get_text()
                date_match = re.search(r'Data publikacji:\s*(.+?)(?:\n|$)', text)
                if date_match:
                    date_text = date_match.group(1).strip()
            else:
                date_text = ''

            if date_text:
                date_of_publication = date_change_format(date_text)
            else:
                date_of_publication = "no date"
        except:
            date_of_publication = "no date"

        # Tytuł
        try:
            # W HTML: <h1 class="text-gold margintop10">Łowca, Aurelia Es</h1>
            title_element = soup.find('h1', class_='text-gold')
            if not title_element:
                title_element = soup.find('h1')

            if title_element:
                title = title_element.get_text(strip=True)
            else:
                title = "no title"
        except:
            title = "no title"

        # Autor
        try:
            # W HTML: <p class="author">Autor: <span>materiały wydawcy</span>
            author = "no author"
            author_p = soup.find('p', class_='author')
            if author_p:
                # Szukamy spana po "Autor:"
                spans = author_p.find_all('span')
                if spans:
                    author = spans[0].get_text(strip=True)
        except:
            author = "no author"

        # Treść artykułu
        try:
            # Główna treść jest w <div class="col-sm-8"> wewnątrz <div class="col-md-10 col-md-push-1 art">
            article_container = soup.find('div', class_='art')

            if article_container:
                # Szukamy głównego kontenera z treścią
                content_div = article_container.find('div', class_='col-sm-8')

                if content_div:
                    # Usuń niepotrzebne elementy
                    for element in content_div.find_all(['script', 'style', 'iframe', 'ins',
                                                         'aside', 'nav', 'header', 'footer',
                                                         'div', 'h2', 'h3']):
                        # Zostawiamy tylko paragrafy
                        if element.name == 'div' and 'buybox' in str(element.get('class', [])):
                            element.decompose()
                        elif element.name == 'div' and 'bb-widget' in str(element.get('id', '')):
                            element.decompose()
                        elif element.name in ['h2', 'h3']:
                            element.decompose()

                    # Wyciągamy tekst tylko z paragrafów
                    paragraphs = content_div.find_all('p')
                    text_parts = []
                    for p in paragraphs:
                        p_text = p.get_text(strip=True)
                        # Pomijamy paragrafy z cenami i innymi metadanymi
                        if p_text and not any(skip in p_text.lower() for skip in
                                            ['autor:', 'wydawnictwo:', 'premiera:', 'liczba stron:',
                                             'udostępnij', 'sprawdź, gdzie kupić']):
                            text_parts.append(p_text)

                    text = ' '.join(text_parts)
                    text = text.replace('\n', ' ').replace('\xa0', ' ')
                    text = re.sub(r'\s+', ' ', text).strip()
                else:
                    text = "no text"
            else:
                text = "no text"
        except:
            text = "no text"

        # Kategoria - z URL
        try:
            # portalkryminalny.pl ma strukturę URL: /aktualnosci/kategoria/tytul
            url_parts = article_link.split('/')
            if 'aktualnosci' in url_parts:
                idx = url_parts.index('aktualnosci')
                if len(url_parts) > idx + 1:
                    category = url_parts[idx + 1]
                    # Kapitalizacja
                    category = category.replace('-', ' ').title()
                else:
                    category = "no category"
            else:
                category = "no category"
        except:
            category = "no category"

        # Tagi
        tags_str = None

        # Linki zewnętrzne
        try:
            article_container = soup.find('div', class_='art')
            if article_container:
                links = [a['href'] for a in article_container.find_all('a', href=True)]
                external_links = [link for link in links
                                if not re.search(r'portalkryminalny\.pl', link)
                                and link.startswith('http')
                                and 'buybox' not in link
                                and 'facebook' not in link
                                and 'twitter' not in link]
                external_links = ' | '.join(external_links) if external_links else None
            else:
                external_links = None
        except:
            external_links = None

        # Zdjęcia
        try:
            images = []

            article_container = soup.find('div', class_='art')
            if article_container:
                # Główne zdjęcie artykułu - w <div class="center-block img-responsive">
                main_img = article_container.find('img', class_='shadow-0')
                if main_img:
                    img_url = main_img.get('src')
                    if img_url:
                        if not img_url.startswith('http'):
                            img_url = 'https://portalkryminalny.pl/' + img_url.lstrip('/')
                        if img_url not in images:
                            images.append(img_url)

                # Wszystkie zdjęcia z treści artykułu
                content_div = article_container.find('div', class_='col-sm-8')
                if content_div:
                    all_imgs = content_div.find_all('img')
                    for img in all_imgs:
                        img_url = img.get('src')
                        if img_url:
                            # Pomijamy małe ikony i loga
                            if not any(x in img_url.lower() for x in ['icon', 'logo', 'avatar', 'btn', 'social']):
                                if img_url not in images:
                                    images.append(img_url)

            has_images = len(images) > 0
            photos_links = ' | '.join(images) if images else None
        except:
            has_images = False
            photos_links = None

        # Filmy
        try:
            has_videos = False
            article_container = soup.find('div', class_='art')
            if article_container:
                iframes = article_container.find_all('iframe', src=True)
                has_videos = len(iframes) > 0
        except:
            has_videos = False

        result = {
            "Link": article_link,
            "Data publikacji": date_of_publication,
            "Tytuł artykułu": title.replace('\xa0', ' '),
            "Tekst artykułu": text,
            "Autor": author,
            "Kategoria": category,
            "Tagi": tags_str,
            "Linki zewnętrzne": external_links,
            "Zdjęcia/Grafika": has_images,
            "Filmy": has_videos,
            "Linki do zdjęć": photos_links
        }

        all_results.append(result)

    except Exception as e:
        errors.append(article_link)


#%% main execution

if __name__ == "__main__":
    print("\n" + "="*60)
    print("SCRAPER PORTALKRYMINALNY.PL")
    print("Portal o literaturze kryminalnej")
    print("="*60 + "\n")

    # KROK 1: Pobierz linki do artykułów
    article_links = get_all_article_links()

    if not article_links:
        print("Nie znaleziono artykułów!")
        exit(1)

    # KROK 2: Scrapuj artykuły
    all_results = []
    errors = []

    print("="*60)
    print("KROK 2: Scraping artykułów")
    print("="*60 + "\n")

    max_workers = 10
    print(f"Używam {max_workers} równoległych wątków\n")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(dictionary_of_article, article_links), total=len(article_links)))

    # KROK 3: Zapisz wyniki
    timestamp = datetime.today().date()

    print(f"\n{'='*60}")
    print("KROK 3: Zapisywanie wyników")
    print("="*60)

    # JSON
    json_file = f'portalkryminalny_{timestamp}.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2, default=str)
    print(f"  ✓ {json_file}")

    # Excel
    excel_file = f"portalkryminalny_{timestamp}.xlsx"
    df = pd.DataFrame(all_results)
    with pd.ExcelWriter(excel_file,
                       engine='xlsxwriter',
                       engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
        df.to_excel(writer, 'Posts', index=False)
    print(f"  ✓ {excel_file}")

    # RAPORT KOŃCOWY
    print(f"\n{'='*60}")
    print("RAPORT KOŃCOWY")
    print("="*60)
    print(f"Pobranych artykułów: {len(all_results)}")
    print(f"Błędów: {len(errors)}")

    if errors and len(errors) <= 10:
        print(f"\nLinki z błędami:")
        for error_link in errors:
            print(f"  - {error_link}")
    elif errors:
        print(f"\nLinki z błędami (pierwsze 10):")
        for error_link in errors[:10]:
            print(f"  - {error_link}")
        print(f"  ... i {len(errors) - 10} więcej")

    print(f"\n{'='*60}")
    print("GOTOWE!")
    print("="*60 + "\n")